In [6]:
import os
import itertools
import codecs
import re
import datetime
import cv2
import matplotlib.pyplot as plt
import numpy as np
from scipy import ndimage
import pylab
from keras import backend as K
from keras.layers.convolutional import Conv2D, MaxPooling2D
from keras.layers import Input, Dense, Activation,Dropout,Flatten
from keras.layers import Reshape, Lambda
from keras.layers.merge import add, concatenate
from keras.models import Model
from keras.layers.recurrent import GRU
from keras.optimizers import SGD
from keras.utils.data_utils import get_file
from keras.preprocessing import image
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
import keras.callbacks

In [7]:
class cood():
    
    def __init__(self,x,y,w,h,name):
        self.x = x
        self.y = y
        self.w = w
        self.h = h
        self.name = name
    
    def getX(self):
        return self.x
    
    def getY(self):
        return self.y
    
    def getW(self):
        return self.w
    
    def getH(self):
        return self.h
    
    def getName(self):
        return self.name
        
class DetectFace():
    
    def __init__(self,img):
        self.img = cv2.imread(img)
        self.cascade_file_url ='cascade/face.xml'
        self.folder = "data/train_set"
        self.model = Sequential()
        self.target_size = (64, 64)
        self.img_cood = []
        self.prediction = []
        
    def print_face(self,img):
        cv2.imshow('face',img)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    def plt_img(self,img):
        plt.imshow(img)
    
    def build_model(self):
        input_shape=(64,64,3)
        num_of_class = len(os.listdir(self.folder))
        self.model.add(Conv2D(32, 3, padding='same', activation='relu', input_shape=input_shape))
        self.model.add(Conv2D(32, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.25))
        
        self.model.add(Conv2D(64, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.5))
        
        self.model.add(Conv2D(128, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.4))
        
        self.model.add(Conv2D(256, 3, padding='same', activation='relu'))
        self.model.add(MaxPooling2D(pool_size = (2,2)))
        self.model.add(Dropout(0.3))
        
        self.model.add(Flatten())
        self.model.add(Dense(512, activation='relu'))
        self.model.add(Dropout(0.55))
        self.model.add(Dense(num_of_class, activation='softmax'))
        
        self.model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
    
    def predict_class(self,image):
        face = cv2.resize(image,self.target_size)
        input_data = face.reshape((-1, face.shape[0], face.shape[1], face.shape[2]))
        ans = self.model.predict(input_data)
        mx = -1
        pos= -1
        for i in range(len(ans[0])):
            if(mx<ans[0][i]):
                mx=ans[0][i]
                pos= i 
        return pos
    
    def load_model(self):
        self.model.load_weights("weights.h5")
    
    def cvtNum(self,data):
        fold = os.listdir(self.folder)
        for i in range(len(fold)):
            if(i==data):
                return fold[i]
    
    def detect_face(self):
        face_cascade = cv2.CascadeClassifier(self.cascade_file_url)
        dt = cv2.cvtColor(self.img.copy(), cv2.COLOR_BGR2GRAY)
        aces = face_cascade.detectMultiScale(dt, 1.3, 5)
        
        for (x,y,w,h) in aces:  
            fc = self.img[y:y+h,x:x+w]
            fc = cv2.fastNlMeansDenoisingColored(fc,None,10,10,7,21)
#             self.print_face(fc)
            data = self.predict_class(fc)
            self.img_cood.append(cood(x,y,w,h,self.cvtNum(data)))
    
    def get_ages(self):
        for points in self.img_cood:
            x = points.getX()
            y = points.getY()
            h = points.getH()
            w = points.getW()
            name = points.getName()
            image = cv2.rectangle(self.img,(x,y),(x+w,y+h),(250,250,25), 2)
            image = cv2.putText(image, "10-19", (x+15,y-10),cv2.FONT_HERSHEY_SIMPLEX, 1, (250,250,0), 1, cv2.LINE_AA)
#             image = cv2.resize(image,(int(image.shape[0]*0.9),int(image.shape[1]*0.9)))
        self.print_face(image)

In [8]:
if __name__ == "__main__":
    df = DetectFace('sample.png')
    df.build_model()
    df.load_model()
    df.detect_face()
    df.get_ages()